In [1]:
import tensorflow as tf

E:\Anaconda3_5_0_0\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

In [10]:
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import standard_ops
from tensorflow.python.framework import ops
from tensorflow.python.framework import common_shapes
from tensorflow.python.framework import tensor_shape

In [31]:
class MyLayer(tf.keras.layers.Layer):
    def __init__(self,output_dim,**kwargs):
        self.output_dim = output_dim
        super(MyLayer,self).__init__(**kwargs)
    
    def build(self,input_shape):
#         print(input_shape)
        shape = tf.TensorShape((input_shape[1],self.output_dim))
#         print(shape)
        self.kernel = self.add_weight(name = 'weight',shape = shape,initializer=tf.random_uniform_initializer,trainable=True)
        super(MyLayer,self).build(input_shape)
        
    def call(self,inputs):
        
        inputs = ops.convert_to_tensor(inputs,dtype=tf.float32)
        rank = common_shapes.rank(inputs)
        if rank>2:
            return standard_ops.tensordot(inputs, self.kernel, [[rank - 1], [0]])
        else:
            return gen_math_ops.mat_mul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        input_shape = tensor_shape.TensorShape(input_shape)
        input_shape = input_shape.with_rank_at_least(2)
#         print(input_shape)
#          shape = tf.TensorShape((input_shape[1],self.output_dim))
        if input_shape[-1].value is None:
            raise ValueError('The innermost dimension of input_shape must be defined, but saw: %s'
                             % input_shape)
        input_shape = input_shape[:-1].concatenate(self.output_dim)
        print(input_shape)
        return input_shape
    
    def get_config(self):
        base_config = super(MyLayer,self).get_config()
        base_config['output_dim'] = self.output_dim
    
    @classmethod
    def from_config(cls,config):
        return cls(**config)

In [32]:
model = tf.keras.Sequential([tf.keras.layers.Dense(128,activation='relu'),
                            MyLayer(10),
                             tf.keras.layers.Activation('softmax')
                            ])

model.compile(optimizer = tf.keras.optimizers.RMSprop(0.01),loss = tf.losses.softmax_cross_entropy,metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)

(?, 28, 10)
Epoch 1/5


InvalidArgumentError: Expected size[0] in [0, 0], but got 1
	 [[{{node loss_7/output_1_loss/softmax_cross_entropy_loss/xentropy/Slice_1}} = Slice[Index=DT_INT32, T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss_7/output_1_loss/softmax_cross_entropy_loss/xentropy/Shape_2, loss_7/output_1_loss/softmax_cross_entropy_loss/xentropy/Slice_2/size, loss_7/output_1_loss/softmax_cross_entropy_loss/xentropy/Slice/size)]]
	 [[{{node loss_7/mul/_551}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_401_loss_7/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]